##first upload data auto dataset and unzip of it

In [3]:
from google.colab import files
uploade = files.upload()

Saving dataset.zip to dataset.zip


In [4]:
!unzip dataset.zip -d /content/dataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Acura_ILX_2020_25_17_200_24_4_70_55_182_24_FWD_5_4_4dr_TRM.jpg  
  inflating: /content/dataset/Acura_ILX_2020_25_17_200_24_4_70_55_182_24_FWD_5_4_4dr_UBV.jpg  
  inflating: /content/dataset/Acura_ILX_2020_25_17_200_24_4_70_55_182_24_FWD_5_4_4dr_UZf.jpg  
  inflating: /content/dataset/Acura_ILX_2020_25_17_200_24_4_70_55_182_24_FWD_5_4_4dr_XMJ.jpg  
  inflating: /content/dataset/Acura_ILX_2020_25_17_200_24_4_70_55_182_24_FWD_5_4_4dr_XuG.jpg  
  inflating: /content/dataset/Acura_ILX_2020_25_17_200_24_4_70_55_182_24_FWD_5_4_4dr_XzS.jpg  
  inflating: /content/dataset/Acura_ILX_2020_25_17_200_24_4_70_55_182_24_FWD_5_4_4dr_yOb.jpg  
  inflating: /content/dataset/Acura_MDX_2011_42_18_300_37_6_78_68_191_16_AWD_7_4_SUV_ARU.jpg  
  inflating: /content/dataset/Acura_MDX_2011_42_18_300_37_6_78_68_191_16_AWD_7_4_SUV_AWZ.jpg  
  inflating: /content/dataset/Acura_MDX_2011_42_18_300_37_6_78_68_191_16_AWD_7_4_SUV_AYx.jpg  

In [5]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

In [6]:
source = Path("/content/dataset")
IMG_EXT = {".jpg",".jpeg"}

In [ ]:
## seperate the label of dataset to form dataframe of brand,model and years of cars with their label and Path :

In [8]:
def parse(p: Path):
  parts = p.stem.split("_")
  brand = parts[0] if len(parts) > 0 else "Unknown"
  model = parts[1]  if len(parts) > 1 else "Unknown"
  year = parts[2]   if len(parts) > 2 and parts[2].isdigit() else "Unknown"
  group = f"{brand}_{model}_{year}"
  return { "brand": brand, "model": model, "year": year, "group": group}


In [9]:
files = [p for p in source.rglob("*") if p.suffix.lower() in IMG_EXT]
df = pd.DataFrame(parse(p) for p in files)
df["source"] = [str(p) for p in files]
print(df.head())


          brand   model  year                   group  \
0         Acura     ILX  2016          Acura_ILX_2016   
1  Aston Martin    DB11  2019  Aston Martin_DB11_2019   
2          Audi      Q5  2012            Audi_Q5_2012   
3  Aston Martin     DBS  2019   Aston Martin_DBS_2019   
4          Audi  e-tron  2019        Audi_e-tron_2019   

                                              source  
0  /content/dataset/Acura_ILX_2016_27_17_200_24_4...  
1  /content/dataset/Aston Martin_DB11_2019_241_20...  
2  /content/dataset/Audi_Q5_2012_35_18_210_20_4_7...  
3  /content/dataset/Aston Martin_DBS_2019_304_21_...  
4  /content/dataset/Audi_e-tron_2019_74_20_400_na...  


In [ ]:
splitting and shuffling of data for training and validation :

In [10]:
split = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(split.split(df, groups=df["group"]))

train_df = df.iloc[train_idx].reset_index(drop=True)
val_df   = df.iloc[val_idx].reset_index(drop=True)

print(f"Total: {len(df)} | Train: {len(train_idx)} | Val: {len(val_idx)}")
print(val_df.head())

Total: 5169 | Train: 4251 | Val: 918
          brand model  year                   group  \
0  Aston Martin  DB11  2019  Aston Martin_DB11_2019   
1          Audi    Q5  2012            Audi_Q5_2012   
2         Acura   RDX  2019          Acura_RDX_2019   
3          Audi    A7  2012            Audi_A7_2012   
4         Acura   MDX  2017          Acura_MDX_2017   

                                              source  
0  /content/dataset/Aston Martin_DB11_2019_241_20...  
1  /content/dataset/Audi_Q5_2012_35_18_210_20_4_7...  
2  /content/dataset/Acura_RDX_2019_39_19_270_20_4...  
3  /content/dataset/Audi_A7_2012_59_18_310_30_6_7...  
4  /content/dataset/Acura_MDX_2017_44_18_290_35_6...  


In [ ]:
mapping and labeling features of autos in dataset :

In [34]:
brand2id = {b: i for i, b in enumerate(sorted(train_df["brand"].unique()))}
model2id = {m: i for i, m in enumerate(sorted(train_df["model"].unique()))}
year2id  = {y: i for i, y in enumerate(sorted(train_df["year"].unique()))}

train_df["brand_id"] = train_df["brand"].map(brand2id)
train_df["model_id"] = train_df["model"].map(model2id)
train_df["year_id"]  = train_df["year"].map(year2id)

val_df["brand_id"] = val_df["brand"].map(brand2id)
val_df["model_id"] = val_df["model"].map(model2id)
val_df["year_id"]  = val_df["year"].map(year2id)

n_brand = len(brand2id)
n_model = len(model2id)
n_year  = len(year2id)

print(brand2id)
print(model2id)
print(year2id)


{'Acura': 0, 'Alfa Romeo': 1, 'Aston Martin': 2, 'Audi': 3}
{'4C': 0, '4C Spider': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'A6': 5, 'A7': 6, 'A8': 7, 'DB11': 8, 'DBS': 9, 'Giulia': 10, 'ILX': 11, 'MDX': 12, 'NSX': 13, 'Q3': 14, 'Q5': 15, 'Q7': 16, 'Q8': 17, 'R8': 18, 'RDX': 19, 'RLX': 20, 'Stelvio': 21, 'TLX': 22, 'TT': 23, 'Vanquish': 24, 'Vantage': 25, 'e-tron': 26}
{'2003': 0, '2004': 1, '2005': 2, '2009': 3, '2010': 4, '2011': 5, '2012': 6, '2013': 7, '2014': 8, '2015': 9, '2016': 10, '2017': 11, '2019': 12, '2020': 13}


In [14]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, Model
import tensorflow as tf


In [ ]:
##read and preprocess of images such as RGB values , size and batch and filter NAN values :

In [15]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def load_image(source, brand, model, year):
    img = tf.io.read_file(source)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, IMG_SIZE)
    labels = {"brand": brand, "model": model, "year": year}
    return img, labels

train_ds = tf.data.Dataset.from_tensor_slices((
    train_df["source"].values,
    train_df["brand_id"].values,
    train_df["model_id"].values,
    train_df["year_id"].values,
)).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Drop rows with NaN values in year_id from val_df
val_df_cleaned = val_df.dropna(subset=['year_id']).reset_index(drop=True)

val_ds = tf.data.Dataset.from_tensor_slices((
    val_df_cleaned["source"].values,
    val_df_cleaned["brand_id"].values,
    val_df_cleaned["model_id"].values,
    val_df_cleaned["year_id"].values,
)).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),                 # flip images randomly
    layers.RandomRotation(0.1),                      # rotate ±10%
    layers.RandomZoom(0.1),                          # zoom in/out ±10%
    layers.RandomBrightness(factor=0.2),             # change brightness ±20%
], name="data_augmentation")

In [ ]:
##loading a pretrained EfficientNet-B0 convolutional backbone and freezing it so its weights are not updated during training :

In [16]:
base = EfficientNetB0(include_top=False, input_shape=(*IMG_SIZE, 3), weights="imagenet")
base.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [17]:
inp = layers.Input(shape=(*IMG_SIZE, 3), name="image")
x = data_augmentation(inp)
x = tf.keras.applications.efficientnet.preprocess_input(inp)
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)

In [31]:
brand_out = layers.Dense(n_brand, activation="softmax", name="brand")(x)
model_out = layers.Dense(n_model, activation="softmax", name="model")(x)
year_out  = layers.Dense(n_year, activation="softmax", name="year")(x)
print(brand_out)
print(model_out)
print(year_out)

<KerasTensor shape=(None, 4), dtype=float32, sparse=False, ragged=False, name=keras_tensor_253>
<KerasTensor shape=(None, 27), dtype=float32, sparse=False, ragged=False, name=keras_tensor_254>
<KerasTensor shape=(None, 14), dtype=float32, sparse=False, ragged=False, name=keras_tensor_255>


In [19]:
model = Model(inputs=inp, outputs=[brand_out, model_out, year_out])

In [20]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(1e-3),
    loss={
        "brand": "sparse_categorical_crossentropy",
        "model": "sparse_categorical_crossentropy",
        "year":  "sparse_categorical_crossentropy",
    },
    metrics={
        "brand": ["sparse_categorical_accuracy"],
        "model": ["sparse_categorical_accuracy"],
        "year":  ["sparse_categorical_accuracy"],
    }
)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image (InputLayer)  │ (None, 224, 224,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 7, 7,      │  4,049,571 │ image[0][0]       │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnetb0[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ brand (Dense)       │ (None, 4)         │      5,124 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ model (Dense)       │ (None, 27)        │     34,587 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ year (Dense)        │ (None, 14)        │     17,934 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,107,216 (15.67 MB)

 Trainable params: 57,645 (225.18 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
##inspect NAN data in training and validation data which show some missing in year head :

In [21]:
for name in ["brand_id","model_id","year_id"]:
    print(name, "NaNs train/val:",
          train_df[name].isna().sum(), val_df[name].isna().sum())

# Show a few problematic rows
bad = val_df[val_df[["brand_id","model_id","year_id"]].isna().any(axis=1)]
print("Bad rows in val:", len(bad))
print(bad[["source","brand","model","year"]].head(10))


brand_id NaNs train/val: 0 0
model_id NaNs train/val: 0 0
year_id NaNs train/val: 0 37
Bad rows in val: 37
                                                source         brand  \
38   /content/dataset/Aston Martin_Vanquish_2006_26...  Aston Martin   
55   /content/dataset/Acura_NSX_2001_84_16_250_30_6...         Acura   
63   /content/dataset/Acura_NSX_2001_84_16_250_30_6...         Acura   
143  /content/dataset/Acura_NSX_2001_84_16_250_30_6...         Acura   
193  /content/dataset/Aston Martin_Vanquish_2006_26...  Aston Martin   
234  /content/dataset/Acura_NSX_2001_84_16_250_30_6...         Acura   
261  /content/dataset/Aston Martin_Vanquish_2006_26...  Aston Martin   
267  /content/dataset/Acura_NSX_2001_84_16_250_30_6...         Acura   
282  /content/dataset/Acura_NSX_2002_89_17_250_30_6...         Acura   
302  /content/dataset/Acura_NSX_2002_89_17_250_30_6...         Acura   

        model  year  
38   Vanquish  2006  
55        NSX  2001  
63        NSX  2001  
143       NS

In [ ]:
##compile and run model as EfficientNetB0-based Multi-Task Image Classifier with Frozen Feature Extractor :

In [22]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss={
        "brand": "sparse_categorical_crossentropy",
        "model": "sparse_categorical_crossentropy",
        "year":  "sparse_categorical_crossentropy",
    },
    metrics=["accuracy"]
)

In [23]:
EPOCHS = 10

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss={
        "brand": "sparse_categorical_crossentropy",
        "model": "sparse_categorical_crossentropy",
        "year":  "sparse_categorical_crossentropy",
    },
    metrics={
        "brand": "sparse_categorical_accuracy",
        "model": "sparse_categorical_accuracy",
        "year":  "sparse_categorical_accuracy",
    }
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_brand_sparse_categorical_accuracy",
                                     mode="max",patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
        mode="min", factor=0.5, patience=2,min_delta=1e-4, min_lr=1e-6,
        verbose=1)
]

hist = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1,
)


Epoch 1/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 400s 3s/step - brand_loss: 1.0109 - brand_sparse_categorical_accuracy: 0.5957 - loss: 6.4245 - model_loss: 3.0225 - model_sparse_categorical_accuracy: 0.1213 - year_loss: 2.3911 - year_sparse_categorical_accuracy: 0.1565 - val_brand_loss: 0.8792 - val_brand_sparse_categorical_accuracy: 0.5698 - val_loss: 6.4169 - val_model_loss: 2.9286 - val_model_sparse_categorical_accuracy: 0.1271 - val_year_loss: 2.5965 - val_year_sparse_categorical_accuracy: 0.1442 - learning_rate: 0.0010
Epoch 2/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 438s 3s/step - brand_loss: 0.6915 - brand_sparse_categorical_accuracy: 0.7211 - loss: 5.0569 - model_loss: 2.2698 - model_sparse_categorical_accuracy: 0.3459 - year_loss: 2.0955 - year_sparse_categorical_accuracy: 0.2586 - val_brand_loss: 0.8422 - val_brand_sparse_categorical_accuracy: 0.6118 - val_loss: 6.1460 - val_model_loss: 2.6738 - val_model_sparse_categorical_accuracy: 0.1952 - val_year_loss: 2.6146 - val_year_sparse_categorical

In [ ]:
##Run model with Fine-Tuned EfficientNetB0-based on  Multi-Head Image Classifier :

In [ ]:
# Unfreeze last N blocks or all layers:
base.trainable = True

#  re-freeze early layers:
for layer in base.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),  # lower LR for fine-tuning
    loss={
        "brand": "sparse_categorical_crossentropy",
        "model": "sparse_categorical_crossentropy",
        "year":  "sparse_categorical_crossentropy",
    },
    metrics={
        "brand": "sparse_categorical_accuracy",
        "model": "sparse_categorical_accuracy",
        "year":  "sparse_categorical_accuracy",
    }
)

hist_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks,
    verbose=1,
)


Epoch 1/5
133/133 ━━━━━━━━━━━━━━━━━━━━ 431s 3s/step - brand_loss: 0.5853 - brand_sparse_categorical_accuracy: 0.7849 - loss: 4.1092 - model_loss: 1.6330 - model_sparse_categorical_accuracy: 0.5369 - year_loss: 1.8908 - year_sparse_categorical_accuracy: 0.3511 - val_brand_loss: 0.6335 - val_brand_sparse_categorical_accuracy: 0.7650 - val_loss: 5.5670 - val_model_loss: 2.0806 - val_model_sparse_categorical_accuracy: 0.3757 - val_year_loss: 2.8397 - val_year_sparse_categorical_accuracy: 0.1407 - learning_rate: 1.0000e-04
Epoch 2/5
133/133 ━━━━━━━━━━━━━━━━━━━━ 379s 3s/step - brand_loss: 0.3320 - brand_sparse_categorical_accuracy: 0.8814 - loss: 3.0794 - model_loss: 1.1191 - model_sparse_categorical_accuracy: 0.6951 - year_loss: 1.6283 - year_sparse_categorical_accuracy: 0.4394 - val_brand_loss: 0.5210 - val_brand_sparse_categorical_accuracy: 0.8195 - val_loss: 5.3739 - val_model_loss: 1.9855 - val_model_sparse_categorical_accuracy: 0.4211 - val_year_loss: 2.8563 - val_year_sparse_categoric

In [ ]:
model.save("auto_image.keras")   #save the model to inference on test and real dataset 

In [52]:
import tensorflow as tf

# Load the saved model
loaded_model = tf.keras.models.load_model("auto_image.keras")

# You can now use the loaded_model for predictions
# For example, if you have new images in a dataset called new_images_ds:
# predictions = loaded_model.predict(new_images_ds)

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
model.save("/content/drive/MyDrive/auto_image.keras")